In [1]:
# =============================
# 1. Imports and Configuration
# =============================
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.measure import label, regionprops, regionprops_table, moments_hu, shannon_entropy
from skimage.feature import local_binary_pattern
from skimage.morphology import remove_small_objects
from scipy.stats import skew, kurtosis
from scipy.spatial import Delaunay
from shapely.geometry import Polygon, MultiPoint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', None)

In [2]:
# =============================
# 2. Feature Extraction Function
# =============================
def extract_features_from_roi(roi_path, roi_crop_dir, n_fourier=3, alpha_value=0.05):
    """Extract shape, texture, and contour-based features from an ROI."""
    try:
        # Load and binarize the mask
        mask = imread(roi_path)
        mask = (mask > 0).astype(np.uint8)

        # Match corresponding grayscale crop
        crop_filename = os.path.basename(roi_path).replace("_mask.tiff", "_crop.tiff")
        crop_path = os.path.join(roi_crop_dir, crop_filename)
        if not os.path.exists(crop_path):
            print(f"Crop not found: {crop_path}")
            return None
        roi_image = imread(crop_path)

        # Label and extract shape descriptors
        labeled = label(mask)
        props_table = regionprops_table(labeled, properties=[
            'area', 'perimeter', 'eccentricity', 'solidity', 'extent',
            'orientation', 'major_axis_length', 'minor_axis_length', 'centroid'
        ])
        props = regionprops(labeled)
        if len(props) == 0:
            print("No props found.")
            return None
        region = props[0]
        convexity = region.convex_area / region.area if region.area > 0 else 0

        # Extract contour and compute Alpha Shape
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if len(contours) == 0:
            print("No contours found.")
            return None
        contour = contours[0].squeeze()
        if contour.ndim != 2 or contour.shape[0] < 3:
            print("Invalid contour shape.")
            return None
        points = np.array(contour)

        def compute_alpha_shape(points, alpha=1.0):
            if len(points) < 4:
                return MultiPoint(points).convex_hull
            tri = Delaunay(points)
            edges = set()
            for ia, ib, ic in tri.simplices:
                pa, pb, pc = points[ia], points[ib], points[ic]
                a = np.linalg.norm(pb - pa)
                b = np.linalg.norm(pc - pb)
                c = np.linalg.norm(pa - pc)
                s = (a + b + c) / 2
                area = np.sqrt(s * (s - a) * (s - b) * (s - c))
                if area == 0:
                    continue
                circum_r = (a * b * c) / (4.0 * area)
                if circum_r < 1.0 / alpha:
                    edges.update([frozenset((ia, ib)), frozenset((ib, ic)), frozenset((ic, ia))])
            if not edges:
                return MultiPoint(points).convex_hull
            unique_indices = set(idx for edge in edges for idx in edge)
            boundary_coords = [points[idx] for idx in unique_indices]
            return Polygon(boundary_coords)

        try:
            alpha_poly = compute_alpha_shape(points, alpha=alpha_value)
            Aa = alpha_poly.area
            Pa = alpha_poly.length
            Ca = 4 * np.pi * Aa / (Pa ** 2) if Pa > 0 else 0
        except Exception as e:
            print("Alpha shape failed:", e)
            Aa, Pa, Ca = 0, 0, 0

        # Convex Hull features
        hull_points = cv2.convexHull(points)
        hull_poly = Polygon(hull_points.squeeze())
        Ah = hull_poly.area
        Ph = hull_poly.length
        Ch = 4 * np.pi * Ah / (Ph ** 2) if Ph > 0 else 0
        area_ratio = Ah / Aa if Aa > 0 else 0

        # Hu Moments (log-transformed)
        hu = moments_hu(region.moments_normalized)
        hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-12)

        # Fourier Descriptors
        complex_contour = contour[:, 0] + 1j * contour[:, 1]
        fourier_result = np.fft.fft(complex_contour)
        fourier_result -= fourier_result[0]
        if np.abs(fourier_result[1]) < 1e-12:
            print("Degenerate normalization factor")
            return None
        amplitudes = np.abs(fourier_result)[:n_fourier]
        phases = np.angle(fourier_result)[:n_fourier]

        # Local Binary Pattern (LBP)
        lbp = local_binary_pattern(roi_image, P=8, R=1, method="uniform")
        lbp_mean = lbp.mean()
        lbp_std = lbp.std()

        # Intensity Features
        intensity_mean = roi_image.mean()
        intensity_std = roi_image.std()
        intensity_skew = skew(roi_image.ravel())
        intensity_kurt = kurtosis(roi_image.ravel())
        intensity_entropy = shannon_entropy(roi_image)

        # Aggregate features
        data = {"filename": os.path.basename(roi_path)}
        for k, v in props_table.items():
            data[k] = v[0]
        for i, val in enumerate(hu_log):
            data[f"hu_{i+1}"] = val
        for i, val in enumerate(amplitudes):
            data[f"fd_amp_{i+1}"] = val
        for i, val in enumerate(phases):
            data[f"fd_phase_{i+1}"] = val
        data.update({
            "lbp_mean": lbp_mean,
            "lbp_std": lbp_std,
            "intensity_mean": intensity_mean,
            "intensity_std": intensity_std,
            "intensity_skew": intensity_skew,
            "intensity_kurt": intensity_kurt,
            "intensity_entropy": intensity_entropy,
            "convexity": convexity,
            "alpha_area": Aa,
            "alpha_perimeter": Pa,
            "Ca": Ca,
            "hull_area": Ah,
            "hull_perimeter": Ph,
            "Ch": Ch,
            "area_ratio": area_ratio,
        })
        return data

    except Exception as e:
        print(f"Error processing {roi_path}: {e}")
        return None


In [3]:
# =============================
# 3. Process All ROIs in Directory
# =============================
roi_dir = "/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data/rois"
roi_crop_dir = "/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data/rois_crops"

min_area = 0
max_area = np.inf
all_features = []

for fname in os.listdir(roi_dir):
    if fname.endswith(".tiff"):
        roi_path = os.path.join(roi_dir, fname)
        mask = imread(roi_path) > 0
        area = np.sum(mask)
        if min_area <= area <= max_area:
            features = extract_features_from_roi(roi_path, roi_crop_dir)
            if features:
                all_features.append(features)
                print("Processed file:", fname)


Processed file: 010-0790-mon_ROI_515_mask.tiff
Processed file: 007-0845-mon_ROI_008_mask.tiff
Processed file: 005-1035-mon_ROI_053_mask.tiff
Processed file: 010-0896-mon_ROI_004_mask.tiff
Processed file: 009-3347-mon_ROI_053_mask.tiff
Processed file: 009-3229-mon_ROI_097_mask.tiff
Processed file: 010-0119-mon_ROI_156_mask.tiff
Processed file: 005-1071-mon_ROI_017_mask.tiff
Processed file: 005-1228-mon_ROI_028_mask.tiff
Processed file: 008-2972-mon_ROI_053_mask.tiff
Processed file: 009-3220-mon_ROI_318_mask.tiff
Processed file: 005-0447-mon_ROI_035_mask.tiff
Processed file: 010-0790-mon_ROI_460_mask.tiff
Processed file: 010-0790-mon_ROI_594_mask.tiff
Processed file: 009-9654-mon_ROI_106_mask.tiff
Processed file: 007-0173-mon_ROI_028_mask.tiff
Processed file: 010-0790-mon_ROI_317_mask.tiff
Processed file: 009-3266-mon_ROI_137_mask.tiff
Processed file: 010-0119-mon_ROI_359_mask.tiff
Processed file: 009-3337-mon_ROI_029_mask.tiff
Processed file: 009-9703-mon_ROI_313_mask.tiff
Processed fil

In [ ]:
# =============================
# 4. Save Features to CSV
# =============================
df = pd.DataFrame(all_features)
df.to_csv("roi_shape_features.csv", index=False)
print(f"Saved {len(df)} feature rows to 'roi_shape_features.csv'")
